In [ ]:
# imports #
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import socket


In [ ]:

# Parameters #
# number of subjects
num=13
# root
if "biowulf" in socket.gethostname():
    root= "/data/NIMH_SFIM/handwerkerd/ComplexMultiEcho1/Data/GroupResults/ROIstats/"
    outdir=root
else:
    root="/Volumes/NIMH_SFIM/handwerkerd/ComplexMultiEcho1/Data/GroupResults/ROIstats/"
    outdir="/Users/handwerkerd/Documents/Presentations/HBM2022/"
# all GLMs
glms = ["e2_mot_CSF","OC_mot_CSF","orthtedana_mot_csf", "combined_regressors"]
# glms=["combined_regressors","e2_mot_CSF","OC_mot","OC_mot_CSF","orthtedana_mot","orthtedana_mot_csf","septedana_mot","septedana_mot_csf"]
colors=["pink","red","blue","green","yellow","orange","brown","purple"]
# all ROIs
rois=['L V1','R V1','L A1','R A1','lPrecuneus','rPrecuneus','lIFG','rIFG','lSTS','rSTS','lMTG', 'rMTG']
#rois=["lCerebrellum","lHippocampus","rCerebellum","rHippocampus","lCuneus","lIFG","l Middle Occipital","L latFusiform","lSPG","lPrecuneus","lITG","lMTG","L V1","lITS","lSTS","L A1","rCuneus","rIFG","r Middle Occipital","R latFusiform","rSPG","rPrecuneus","rITG","rMTG","R V1","rITS","rSTS","R A1"] 
# colors
viridis=cm.get_cmap('viridis', len(rois))
color_cycler = viridis(np.linspace(0,1,len(rois)))
##############


In [ ]:

# Degrees of freedom across the GLMs
glms = ["e2_mot_CSF","OC_mot_CSF","orthtedana_mot_csf", "combined_regressors"]
dof = pd.read_csv(f"{root}DOF_GLMs_All_group.tsv", sep='\t')
fig = plt.figure(figsize=(5,8))
plt.plot(dof[glms].T)
plt.title("Degrees of Freedom")
plt.show()



In [ ]:
# USED ON OHBM 2022 POSTER
# Degrees of freedom across the GLMs

# From sub-08 OC_mot_CSF:
# initial DF                   : 1035 : 100.0%
# DF used for regs of interest :    5 :   0.5%
# DF used for censoring        :    5 :   0.5%
# DF used for polort           :   15 :   1.4%
# DF used for motion           :   36 :   3.5%
# DF used for ROIPC            :    9 :   0.9%
# total DF used                :   70 :   6.8%
# final DF                     :  965 :  93.2%
# Ignoring censoring, 60 of 1035 DF are used for noise removal. That should be a consistent baseline

glms = ["e2_mot_CSF","OC_mot_CSF","orthtedana_mot_csf", "combined_regressors"]
dof = pd.read_csv(f"{root}DOF_GLMs_All_group.tsv", sep='\t')
baseshift = dof["e2_mot_CSF"]+60
for glm in glms:
    dof[glm] = dof[glm]-baseshift

fig = plt.figure(figsize=(5,5))
ax = fig.subplots()
plt.plot(dof[glms].T)
plt.title("Lost Degrees of Freedom from Noise Removal")
ax.set_xticklabels(['2nd echo: motion+CSF', 'Opt Combined: motion+CSF', 'Tedana: motion+CSF', 'Combined Regressors'])
plt.xticks(rotation = 15) #, fontsize=14)
plt.show()
fig.savefig(f"{outdir}DOF_loss.eps")

In [ ]:
voxels_per_glm=dict()
for gidx, g in enumerate(glms):
    voxels_per_glm[g] = pd.read_csv(f"{root}{g}_Voxels_group.tsv", sep='\t')

    if gidx==0:
        voxelcounts = voxels_per_glm[g]
        voxelcounts_nan0=np.nan_to_num(voxelcounts.to_numpy())
    else:
        tmp = np.nan_to_num(voxels_per_glm[g].to_numpy()) - voxelcounts_nan0
        if tmp.sum() > 0.0:
            raise ValueError("Voxel counts should be the same across GLMs, but are not")
print((voxelcounts[rois]>5).sum())
